In [32]:
import numpy as np
import random


class nn:
    def __init__(self, batchSize=100, feature_layer=None, hidden_layer=list(), output_layer=None, eta=0.1, activationFunctions=dict(), costFunction='rms', seed=None):
        self.random_generator = np.random.RandomState(seed)
        l = [self.sigmoid]*(len(hidden_layer)+1)
        dl = [self.sigmoidDerivative]*(len(hidden_layer)+1)
        for i in activationFunctions.keys():
            if (str.lower(activationFunctions[i]) == 'relu'):
                l[i-1] = self.ReLU
                dl[i-1] = self.ReLUDerivative
            elif (str.lower(activationFunctions[i]) == 'sigmoid'):
                pass
            elif (str.lower(activationFunctions[i]) == 'linear'):
                l[i-1] = self.linear
                dl[i-1] = self.linearDerivative
            elif (str.lower(activationFunctions[i]) == 'tanh'):
                l[i-1] = self.tanh
                dl[i-1] = self.tanhDerivative
            else:
                raise Exception(
                    'only "relu" and "sigmoid" are valid options for activationFunction parameters')

        if (str.lower(costFunction) == 'rms'):
            self.costFunction_ = self.RMS
            self.costFunctionDerivative_ = self.RMSderivative
        elif (str.lower(costFunction) == 'bce'):
            self.costFunction_ = self.BCE
            self.costFunctionDerivative_ = self.BCEderivative
        else:
            raise Exception(
                'only "rms" and "bce" are valid options for costFunction parameters')

        self.activationFunction_ = l
        self.activationDerivative_ = dl
        self.batchSize_ = batchSize
        self.layers = [feature_layer]+hidden_layer+[output_layer]
        self.eta = eta
        self.init_eta_ = self.eta
        self.weights_ = list()
        self.deltaW_ = list()
        self.activations_ = list()
        self.neuronInputs_ = list()
        self.validationAccuracy_ = -1
        self.validation_set_ = None

        if seed:
            np.random.seed(seed)

        # initialize weights and biases
        l = self.layers
        for i in range((len(l)-1)):
            # self.weights_.append(np.random.rand(l[i]+1, l[i+1]))
            self.weights_.append(np.random.normal(
                size=(l[i]+1, l[i+1]), scale=1/np.sqrt(l[i])))
            #self.weights_.append(self.random_generator.normal(size=(l[i]+1, l[i+1]),scale=1/np.sqrt(l[i])))
            self.deltaW_.append(np.zeros((l[i]+1, l[i+1])))
            self.activations_.append(np.zeros(l[i]))
            self.neuronInputs_.append(np.zeros(l[i]))
            # initializing bias to zeros
        for i in self.weights_:
            i[i.shape[0]-1] = np.zeros(i.shape[1])

    def resetDeltaW(self):
        self.deltaW_ = list()
        l = self.layers
        for i in range((len(l)-1)):
            self.deltaW_.append(np.zeros((l[i]+1, l[i+1])))

    def sigmoid(self, l):
        x = l
        # return 1.0 / (1.0 + np.exp(-x))
        for i in range(len(x)):
            if x[i] < 0:
                x[i] = np.exp(x[i]) / (1.0 + np.exp(x[i]))
            else:
                x[i] = 1.0 / (1.0 + np.exp(-x[i]))
        return x

    def sigmoidDerivative(self, x):
        return x*(1.0 - x)

    def ReLU(self, x):
        return x*(x > 0)

    def ReLUDerivative(self, l):
        x = l
        for i in range(len(x)):
            if x[i] == 0.0:
                x[i] = random.random()
                # x[i]=0.0
            elif x[i] > 0.0:
                x[i] = 1.0
            else:
                x[i] = 0.0
        return x

    def linear(self, l):
        return l

    def linearDerivative(self, l):
        return l  # np.ones(l.shape)

    def tanh(self, l):
        return np.tanh(l)

    def tanhDerivative(self, l):
        return 1 - l ** 2

    def RMS(self, x, y):
        return np.average((y-x)**2)

    def RMSderivative(self, x, y):
        return (y-x)

    # binary cross entropy
    def BCE(self, l, y):
        x = np.round(l, 5)
        for i in range(len(x)):
            if y[i] == 1:
                x[i] = -np.log(x[i])
            else:
                x[i] = -np.log(1-x[i])
        return np.sum(x)/len(x)

    def BCEderivative(self, l, y):
        x = np.round(l, 5)
        for i in range(len(x)):
            if y[i] == 1:
                x[i] = -np.log(1/x[i])
            else:
                x[i] = np.log(1/(1-x[i]))
        return x

    def feedForward(self, inputs):
        layer_output = list()
        layer_input = list()
        layer_output.append(inputs)
        layer_input.append(inputs)
        # activations from the first layer is the input
        activations = inputs
        for i in zip(self.weights_, self.activationFunction_):
            # calculating input to the neurons
            neuron_input = np.dot(np.append(activations, 1), i[0])
            # neuron_input=np.dot(np.append(activations,1),i[0].T)
            # neuron_input=np.matmul(np.append(activations,1).T,i[0])
            # passing it through the activation function for the given layer
            neuron_output = i[1](neuron_input)
            # activations for next layer is inputs from previous layer
            activations = neuron_output
            # saving the activations from each layers
            layer_output.append(neuron_output)
            layer_input.append(neuron_input)
        self.activations_ = layer_output
        self.neuronInputs_ = layer_input
        # print(activations)
        # time.sleep(1)
        return activations

    # adaptive learning
    def updateLearningRate(self, epoch):
        self.eta = self.init_eta_/np.sqrt(epoch)

    def converge(self):
        mask = np.array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])
        correct = 0
        for v in self.validation_set_:
            modelOutput = self.feedForward(v[0])
            # class with the maximum activation is our predicted class
            modelOutput = np.array(
                [1 if i == max(modelOutput) else 0 for i in modelOutput])
            # creating a class mask for convinience in confusion matrix
            modelOutput = np.dot(mask, modelOutput)
            targetOutput = int(np.dot(mask, v[1]))
            if modelOutput == targetOutput:
                correct += 1
        accuracy = int(correct*100/len(self.validation_set_))
        print('Validation accuracy: {}%'.format(accuracy))
        if (accuracy == self.validationAccuracy_):
            return True
        elif (self.validationAccuracy_ < accuracy):
            self.validationAccuracy_ = accuracy
        return False

    def train(self, inputs, targets):
        # creating a validation set which will be used in checking convergence
        mask = [random.randint(0, len(inputs)-1) for i in range(4000)]
        self.validation_set_ = [(inputs[i], targets[i]) for i in mask]

        epoch = 1
        while (self.converge() != True):
            # epoch
            # self.updateLearningRate(epoch)
            for iter in range(int(len(targets)/self.batchSize_)):
                sumError = 0
                # batch
                # after each bach reset the deltaW to 0s
                self.resetDeltaW()
                for i in range(self.batchSize_):
                    index = random.randint(0, len(inputs)-1)
                    o = self.feedForward(inputs[index])
                    error = self.costFunctionDerivative_(o, targets[index])
                    self.backPropogation(error)
                    sumError += self.costFunction_(o, targets[index])
                self.gradientDescent()
            print('epoch: {}  Error: {}'.format(epoch, sumError))
            print('learning rate used: {}'.format(self.eta))
            epoch += 1

    def backPropogation(self, error):
        # moving backwards
        for i in range(len(self.weights_)-1, -1, -1):
            delW = error*self.activationDerivative_[i](self.activations_[i+1])
            # if(type(self.activationFunction_[i])==type(self.sigmoid)):
            #     delW=error*self.activationDerivative_[i](self.activations_[i+1])
            # else:
            #     delW=error*self.activationDerivative_[i](self.neuronInputs_[i+1])
            act = (np.append(self.activations_[i], 1)).reshape(
                self.layers[i]+1, 1)
            delW = delW.reshape(1, self.layers[i+1])
            # calculating deltaW for weights and bias of a layer
            self.deltaW_[i] += np.matmul(act, delW)
            # calculating error for the previous layer
            error = np.dot(delW, self.weights_[i].T)
            # dropping error for bias not, no need to backpropogate it has no back connections
            error = np.delete(error, error.shape[1]-1)

    def gradientDescent(self):
        for i in range(len(self.weights_)):
            self.weights_[i] -= (self.deltaW_[i]/self.batchSize_)*self.eta
            # self.weights_[i]+=self.deltaW_[i]*self.eta


In [33]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split

X, y = fetch_openml(
    "mnist_784", version=1, return_X_y=True, as_frame=False, parser="pandas"
)

y = np.array(y, dtype=int)

# split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42)


mod = nn(feature_layer=784, hidden_layer=[89], costFunction='rms', output_layer=10,
         batchSize=500, activationFunctions={1: 'tanh', 2: 'sigmoid'})


y_train = y_train.reshape(-1, 1)
enc = OneHotEncoder(handle_unknown='ignore')
enc.fit(y_train)

y_train = enc.transform(y_train).toarray()

mod.train(X_train, y_train)


Validation accuracy: 11%
epoch: 1  Error: 74.05418592947015
learning rate used: 0.1
Validation accuracy: 85%
epoch: 2  Error: 60.67668371104687
learning rate used: 0.1
Validation accuracy: 87%
epoch: 3  Error: 52.93265047857873
learning rate used: 0.1
Validation accuracy: 89%
epoch: 4  Error: 46.77851936153485
learning rate used: 0.1
Validation accuracy: 89%


In [31]:
def test(inputs, targets):
    # creating a class mask for convinience in confusion matrix
    mask = np.array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])
    # initializing the confusion matrix
    confusionMatrix = np.zeros((10, 10), dtype=int)
    for i in range(len(targets)):
        modelOutput = mod.feedForward(inputs[i])
        # class with the maximum activation is our predicted class
        modelOutput = np.array(
            [1 if i == max(modelOutput) else 0 for i in modelOutput])

        modelOutput = np.dot(mask, modelOutput)
        targetOutput = int(np.dot(mask, targets[i]))
        confusionMatrix[modelOutput][targetOutput] += 1
    # # modelOutput=[round(k) for k in modelOutput]
    # print("----------------confusion matrix----------------")
    # print(confusionMatrix[:10])
    # print("------------------------------------------------")
    # calculating the accuracy
    accuracy = np.trace(confusionMatrix)*100/np.sum(confusionMatrix)
    print('Accuracy: {}%'.format(accuracy))
    return accuracy


y_test = y_test.reshape(-1, 1)
enc = OneHotEncoder(handle_unknown='ignore')
enc.fit(y_test)
y_test = enc.transform(y_test).toarray()

test(X_test, y_test)


Accuracy: 89.92857142857143%


89.92857142857143